In [1]:
import numpy as np
import copy
import dimod
import random
import math
from math import log
import dimod

Given Matrix A (m times n) and a column vector (b) of length m. Find column vector x length n, such that
||Ax - b|| is minimized

To start with, let us consider a case where x is a binary vector taking 0 or 1

In [2]:
#A is a 2x2 matrix b is length 2, x will be a length 2 matrix
A = np.array([[2,1],[1,3]])
b = np.array([1,3])

Let us take qubits values q0 for x0 and q1 for x1 such that x = np.array([x0,x1]). For the sake of the example we know beforehand that the best answer is x = np.array([0,1])

The QUBO formulation based on L2 norm squaring for the example above will be (refer to Eqn 3.5 and 3.6 in my dissertation)

In [3]:
Q = {}
#First the individual coefficients
Q['q0','q0'] = A[0,0]*(A[0,0] - 2*b[0]) +  A[1,0]*(A[1,0] - 2*b[1])
Q['q1','q1'] = A[1,0]*(A[1,0] - 2*b[0]) +  A[1,1]*(A[1,1] - 2*b[1])
#and now for the interaction between q0 and q1
Q['q0','q1'] = 2*(A[0,0]*A[0,1] + A[1,0]*A[1,1])
Q

{('q0', 'q0'): -5, ('q1', 'q1'): -10, ('q0', 'q1'): 10}

Let us run this thing with exact solver

In [4]:
#Use Exactsolver to solve the qubo
sampler = dimod.ExactSolver()
sampleset = sampler.sample_qubo(Q)

In [5]:
#Let's get the 'best' answer from the sampleset object
print("Result is :", sampleset.first.sample)

Result is : {'q0': 0, 'q1': 1}


Which means the result is x = np.array([0,1])

------------------------------------------------<br>
Now let us take a slightly different example which will be somewhat a bulding component of V = WH.<br>
Take a quadratic equation with threee variables x0,x1 and x2 that are binary, not using q0 and q1 (personal choice, you can use it)

The problem is find x = [x2,x1,x0] such that we minimize (2 - x2 - x1 - x0)^2. Here, we know the best answer is for exactly two of the variables to be 1 and the other to be a 0. But let us see how to get a QUBO for it.

Remember that when you exand something like (a + b + c +.. +z)^2 your result is : a^2 + b^2 + ... + z^2 + 2ab + 2ac + 2ad + ... + 2 yz. <br> (where all combinations of the terms are multiplied with each other. N choose 2)

Thus, ( 2 - x2 - x1 - x0) ^ 2 = x0^2 + x1^2 + x2^2 + 2(-x0)(-x1) + 2(-x1)(-x2) + 2(-x0)(-x2) + 2(2)(-x0) +2(2)(-x1) +2(2)(-x2) + 4 <br><br>The signs of each term multiply during their interactions. Also, do you notice the + 2(2)(-x0) +2(2)(-x1) +2(2)(-x2)? This happens because even the standalone 2 is considered as a term for the expansion of this quadratic form.

Okay, since we are dealing with binary variables, x0^2 = x0) (same goes for all of them). Thus the form becomes <br> = 1(1 - 2(2))x0 - 1(1 - 2(2))x1 - 1(1 - 2(2))x2 + 2x0x1 + 2x1x2 + 2x0x2 + 4

In [15]:
#Now for the QUBO, we ignore the constant, which is 4 in this case
#Linear coefficients
Q= {}
Q['x0','x0'] = 1*(1 - 2*(2))
Q['x1','x1'] = 1*(1 - 2*(2))
Q['x2','x2'] = 1*(1 - 2*(2))
#Quadratic coefficients
Q['x0','x1'] = 2
Q['x0','x2'] = 2
Q['x1','x2'] = 2
Q

{('x0', 'x0'): -3,
 ('x1', 'x1'): -3,
 ('x2', 'x2'): -3,
 ('x0', 'x1'): 2,
 ('x0', 'x2'): 2,
 ('x1', 'x2'): 2}

In [16]:
#Use Exactsolver to solve the qubo
sampler = dimod.ExactSolver()
sampleset = sampler.sample_qubo(Q)

In [17]:
print(sampleset)

  x0 x1 x2 energy num_oc.
2  1  1  0   -4.0       1
4  0  1  1   -4.0       1
6  1  0  1   -4.0       1
1  1  0  0   -3.0       1
3  0  1  0   -3.0       1
5  1  1  1   -3.0       1
7  0  0  1   -3.0       1
0  0  0  0    0.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


You can see the top 3 results (any one of them) satisfy the result. (ignore the first column, its just the original order of the result, see sampleset.samples)

------------------------------------------<br>
Now let us go a step further let us solve for a binary vector x =  np.array([x0,x1,x2]), where we need to minimize:<br> (3 + 4x2 - 2x1 -x0)^2. For this problem, logically, we can see that putting x2 = 0 and x1 and x0 as 1 will minimize the expression the most. But let us see how to make a qubo for that.

So, when it comes to expand an algebraic expression that is raised to power two. Terms such as 4x2 or -2x1 are considered single terms. Which means whatever coefficients they have will also 'follow along for the ride'. 

(3 + 4x2 - 2x1 - x0)^2 = (4x2)^2 + (2x1)^2 + (x0)^2 + 2(4x2)(-2x1) + 2(-2x1)(-x0) + 2(4x2)(-x0) + 2(3)(4x2) + 2(3)(-2x1) + 2(3)(-x0) + 9 <br>
Simplifying, and remembering binary variables are idempotent (x0^2 = x0 and so forth) we get<br>

= 4(4 + 2(3))x2 + 2(2 - 2(3))x1 + 1(1 - 2*(3)) - 2(8)x2x0 - 2(4)x2x0 + 2(2)x1x0 + 9

In [18]:
#Now for the QUBO, we ignore the constant, which is 9 in this case
#Linear coefficients
Q = {}
Q['x0','x0'] = 1*(1 - 2*(3))
Q['x1','x1'] = 2*(2 - 2*(3))
Q['x2','x2'] = 4*(4 + 2*(3))
#Quadratic coefficients
Q['x0','x1'] = 2*(2)
Q['x0','x2'] = -2*(4)
Q['x1','x2'] = -2*(8)
Q

{('x0', 'x0'): -5,
 ('x1', 'x1'): -8,
 ('x2', 'x2'): 40,
 ('x0', 'x1'): 4,
 ('x0', 'x2'): -8,
 ('x1', 'x2'): -16}

In [19]:
#Use Exactsolver to solve the qubo
sampler = dimod.ExactSolver()
sampleset = sampler.sample_qubo(Q)

In [21]:
print(sampleset)
sampleset.first.sample

  x0 x1 x2 energy num_oc.
2  1  1  0   -9.0       1
3  0  1  0   -8.0       1
1  1  0  0   -5.0       1
0  0  0  0    0.0       1
5  1  1  1    7.0       1
4  0  1  1   16.0       1
6  1  0  1   27.0       1
7  0  0  1   40.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


{'x0': 1, 'x1': 1, 'x2': 0}

Look at the first row (ignore the first column, its just the original order of the result, see sampleset.samples)